In [2]:
import tensorflow as tf
from keras import layers, optimizers, Sequential
from keras.datasets import fashion_mnist

batches = 128

In [3]:
def preprocess(x, y):
    # 并做归一化处理
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x, y


(x, y), (x_test, y_test) = fashion_mnist.load_data()
db = tf.data.Dataset.from_tensor_slices((x, y))
db = db.map(preprocess).shuffle(10000).batch(batches)

db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.map(preprocess).batch(batches)

db_iter = iter(db)
sample = next(db_iter)
print('batch:', sample[0].shape, sample[1].shape)

2022-11-17 17:13:29.568621: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-17 17:13:29.568755: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-11-17 17:13:29.635184: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Metal device set to: Apple M1 Pro
batch: (128, 28, 28) (128,)


In [4]:
model = Sequential([
    layers.Dense(256, activation=tf.nn.relu),
    # [b,256]--> [b,128]
    layers.Dense(128, activation=tf.nn.relu),
    # [b, 128] --> [b,64]
    layers.Dense(64, activation=tf.nn.relu),
    # [b,64] --> [b,32]
    layers.Dense(32, activation=tf.nn.relu),
    # [b,32] --> [b,10] 输出层
    layers.Dense(10)
])

model.build(input_shape=[None, 28 * 28])
optimizers = optimizers.Adam(learning_rate=1e-3)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               200960    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 10)                330       
                                                                 
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [5]:
acc_test = 0
for epoch in range(5):
    print("---------------------epoch{:2d}---------------------".format(epoch+1))
    for step, (x, y) in enumerate(db):
        x = tf.reshape(x, [-1, 28 * 28])
        with tf.GradientTape() as tape:
            logits = model(x)
            y_onehot = tf.one_hot(y, depth=10)
            loss_mse = tf.reduce_mean(tf.losses.MSE(y_onehot, logits))
            loss_ce = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)
            loss_ce = tf.reduce_mean(loss_ce)

        grads = tape.gradient(loss_ce, model.trainable_variables)
        optimizers.apply_gradients(zip(grads, model.trainable_variables))

        if (step+1) % 100 == 0:
            print('step:{:3d}, loss:{:.5f}'.format(step+1, float(loss_ce), float(loss_mse)))

    total_correct = 0
    total_num = 0
    for x, y in db_test:
        x = tf.reshape(x, [-1, 28 * 28])
        logits = model(x)

        prob = tf.nn.softmax(logits, axis=1)
        pred = tf.argmax(prob, axis=1)
        pred = tf.cast(pred, dtype=tf.int32)
        correct = tf.equal(pred, y)
        correct = tf.reduce_sum(tf.cast(correct, dtype=tf.int32))
        total_correct += int(correct)
        total_num += x.shape[0]
    acc = total_correct / total_num
    if acc > acc_test:
        acc_test = acc
        total_correct = 0
        total_num = 0
        for x, y in db:
            x = tf.reshape(x, [-1, 28 * 28])
            logits = model(x)

            prob = tf.nn.softmax(logits, axis=1)
            pred = tf.argmax(prob, axis=1)
            pred = tf.cast(pred, dtype=tf.int32)
            correct = tf.equal(pred, y)
            correct = tf.reduce_sum(tf.cast(correct, dtype=tf.int32))
            total_correct += int(correct)
            total_num += x.shape[0]
        acc_train = total_correct / total_num
        print('train acc:{:.5f}'.format(acc_train))
    print('text acc:{:.5f}'.format(acc))

---------------------epoch 1---------------------
step:100, loss:0.50941
step:200, loss:0.56438
step:300, loss:0.47671
step:400, loss:0.52274
train acc:0.85217
text acc:0.83680
---------------------epoch 2---------------------
step:100, loss:0.33205
step:200, loss:0.33108
step:300, loss:0.36324
step:400, loss:0.30593
train acc:0.88115
text acc:0.86330
---------------------epoch 3---------------------
step:100, loss:0.26997
step:200, loss:0.46384
step:300, loss:0.35244
step:400, loss:0.27779
train acc:0.88388
text acc:0.86650
---------------------epoch 4---------------------
step:100, loss:0.26404
step:200, loss:0.36499
step:300, loss:0.38806
step:400, loss:0.23430
train acc:0.89573
text acc:0.87530
---------------------epoch 5---------------------
step:100, loss:0.43204
step:200, loss:0.33360
step:300, loss:0.26441
step:400, loss:0.29038
text acc:0.87470
